In [97]:
from functools import partial
import subprocess
import glob
import time
import numpy as np




kiter_executable = "./Release/bin/kiter"
cmdKperiodic = lambda filename : [kiter_executable, "-f", filename,  "-a", "KPeriodicThroughput"]


class BenchmarkResult :
    execution_time = None
    stdout         = None

    def __init__ (self, et = None, stdout = None) :
        self.execution_time = et
        if (type(stdout) == bytes) :
            self.stdout         = stdout.decode("utf-8") 
        else :
            self.stdout         = stdout
            
    def getExecutionTime(self) :
        return self.execution_time
    
    def getComputeTime (self):
        if (not self.stdout) : return None
        for l in self.stdout.split("\n") :
            if "Time" in l :
                return float(l.split(" ")[-1])
    
def runBenchmarkOnce (filename) :
    
    command = cmdKperiodic(filename = filename)
    
    try :
        start_time = time.time()
        result = subprocess.run(command, stdout=subprocess.PIPE,timeout = 3600)
        end_time = time.time()
        executiontime = (end_time  - start_time) * 1000
        output = result.stdout
    except subprocess.TimeoutExpired as e:
        executiontime = None
        output = None
    
    return BenchmarkResult (executiontime  ,  output )

def runBenchmark (filename, it = 10) :
    execution_times = []
    compute_times = []
    for _ in range(it) :
        res = runBenchmarkOnce (filename)
        execution_times += [res.getExecutionTime()]
        compute_times += [res.getComputeTime()]
    if None in execution_times or None in compute_times :
        return None, None
    return np.median(execution_times) ,  np.median(compute_times)

def formatTime (et , rj = 10) : 
    return '{:.2f} ms'.format(et).rjust(rj) if et else '-'.rjust(rj)
    
def runBenchmarks (filelist) :
    colSize = max([len(x) for x in filelist]) + 10

    print("Name".rjust(colSize), "ExecutionTime".rjust(15), "ComputeTime".rjust(15))
    for filename in sorted(filelist) :
        execution_time,compute_time  = runBenchmark(filename)
        print(filename.rjust(colSize),
              formatTime(execution_time,15),
              formatTime(compute_time,15))


benchmarks  = glob.glob('./benchmark/*.xml')
benchmarksS = glob.glob('./benchmark_sized/*.xml')
benchmarksG = glob.glob('./AGB5CSDF/*.xml')

In [98]:
runBenchmarks (benchmarks)

                                              Name   ExecutionTime     ComputeTime
                                ./benchmark/21.xml         6.77 ms         0.11 ms
                      ./benchmark/BlackScholes.xml         8.62 ms         0.46 ms
                              ./benchmark/Echo.xml         8.48 ms         0.29 ms
                              ./benchmark/H264.xml       305.48 ms        33.63 ms
                          ./benchmark/JPEG2000.xml        45.28 ms         3.95 ms
                          ./benchmark/Pdectect.xml        44.87 ms        22.96 ms
          ./benchmark/expansion_paper_norm_sdf.xml         5.95 ms         0.08 ms
               ./benchmark/expansion_paper_sdf.xml         6.30 ms         0.09 ms
                       ./benchmark/init_sample.xml         6.31 ms         0.13 ms
                     ./benchmark/new_benchmark.xml         6.79 ms         0.17 ms
                            ./benchmark/sample.xml         6.35 ms         0.13 ms
    

In [99]:
runBenchmarks (benchmarksS)

                                         Name   ExecutionTime     ComputeTime
          ./benchmark_sized/Black-scholes.xml        10.99 ms         0.86 ms
                   ./benchmark_sized/Echo.xml     31448.85 ms     31431.82 ms
                   ./benchmark_sized/H264.xml      1102.62 ms        75.54 ms
               ./benchmark_sized/JPEG2000.xml      1182.44 ms      1089.12 ms
               ./benchmark_sized/Pdectect.xml        74.78 ms        46.48 ms


In [100]:
runBenchmarks (benchmarksG)

                             Name   ExecutionTime     ComputeTime


FileNotFoundError: [Errno 2] No such file or directory: './Release/bin/kiter': './Release/bin/kiter'